## generate llm input data example!!

user 2의 2025-08-05 관련 데이터 4종을

- LLM 학습용 JSONL 레코드 1줄
- 각 파트별 개별 JSON 파일

으로 저장하는 스크립트

In [19]:
import json
import pandas as pd
from pathlib import Path

# ──────────────────────────────────────────────────────────────────────────
# 0) 공통 설정
# ──────────────────────────────────────────────────────────────────────────
BASE_DIR   = Path("/Users/yujimin/KB AI CHALLENGE/project")
DATA_DIR   = BASE_DIR / "data"
OUTPUT_DIR = BASE_DIR / "llm_input"
OUTPUT_DIR.mkdir(exist_ok=True)

TARGET_USER = 2
TARGET_DATE = pd.to_datetime("2025-08-05").date()   # YYYY-MM-DD

# ──────────────────────────────────────────────────────────────────────────
# 1) 사용자 프로필
# ──────────────────────────────────────────────────────────────────────────
users_df       = pd.read_json(DATA_DIR / "dummy_users.json")
user_profile   = users_df.loc[users_df["userId"] == TARGET_USER].iloc[0].to_dict()

# ──────────────────────────────────────────────────────────────────────────
# 2) 거래 내역
# ──────────────────────────────────────────────────────────────────────────
trades_df            = pd.read_json(DATA_DIR / "dummy_trade_history_v2.json")
trades_df["order_date"] = pd.to_datetime(trades_df["orderTime"]).dt.date

user_trades = trades_df.loc[
    (trades_df["userId"] == TARGET_USER) & (trades_df["order_date"] == TARGET_DATE)
].copy()

# 문자열 변환(ISO8601) → JSON 직렬화 시 pandas Timestamp 문제 방지
user_trades["orderTime"]     = user_trades["orderTime"].astype(str)
user_trades["completionTime"] = user_trades["completionTime"].astype(str)

# ──────────────────────────────────────────────────────────────────────────
# 3) 투자 일지
# ──────────────────────────────────────────────────────────────────────────
journals_df = pd.read_json(DATA_DIR / "dummy_trade_journals_v2.json")

# orderTime 붙이기
journals_with_time = journals_df.merge(
    trades_df[["id", "orderTime"]], left_on="tradeId", right_on="id", how="left"
)
journals_with_time["orderTime"] = pd.to_datetime(journals_with_time["orderTime"])

user_journals = journals_with_time.loc[
    (journals_with_time["userId"] == TARGET_USER) &
    (journals_with_time["orderTime"].dt.date == TARGET_DATE)
].copy()

user_journals["orderTime"] = user_journals["orderTime"].astype(str)

# ──────────────────────────────────────────────────────────────────────────
# 4) 8월 5일 뉴스
# ──────────────────────────────────────────────────────────────────────────
# news_df   = pd.read_csv(BASE_DIR / "news_clean_dedup.csv")
news_0805   = pd.read_csv(BASE_DIR / "/Users/yujimin/KB AI CHALLENGE/project/src/news_2025-08-05_sent_scored.csv")
# news_0805 = news_df.loc[news_df["date"] == "2025.08.05"].copy()

# ──────────────────────────────────────────────────────────────────────────
# 5) 학습용 단일 JSON 레코드 구성 & 저장(JSONL 한 줄)
# ──────────────────────────────────────────────────────────────────────────
train_record = {
    "meta": {
        "user_id": TARGET_USER,
        "datetime": TARGET_DATE.isoformat()
    },
    "user_profile": user_profile,
    "trades":  user_trades.drop(columns=["order_date"]).to_dict(orient="records"),
    "journals": user_journals[
        ["journalId", "tradeId", "orderTime", "reason", "emotion", "behavior", "resultType"]
    ].to_dict(orient="records"),
    "news": news_0805.to_dict(orient="records")
}

with open(OUTPUT_DIR / f"train_user{TARGET_USER}_{TARGET_DATE}.jsonl", "w", encoding="utf-8") as f:
    f.write(json.dumps(train_record, ensure_ascii=False) + "\n")

# ──────────────────────────────────────────────────────────────────────────
# 6) 파트별 개별 JSON 저장(선택)
# ──────────────────────────────────────────────────────────────────────────
with open(OUTPUT_DIR / f"user{TARGET_USER}_profile.json", "w", encoding="utf-8") as f:
    json.dump(user_profile, f, ensure_ascii=False, indent=2)

user_trades.drop(columns=["order_date"]).to_json(
    OUTPUT_DIR / f"trades_user{TARGET_USER}_{TARGET_DATE}.json",
    orient="records", force_ascii=False, indent=2
)

user_journals[
    ["journalId", "tradeId", "orderTime", "reason", "emotion", "behavior", "resultType"]
].to_json(
    OUTPUT_DIR / f"journals_user{TARGET_USER}_{TARGET_DATE}.json",
    orient="records", force_ascii=False, indent=2
)

news_0805.to_json(
    OUTPUT_DIR / f"news_{TARGET_DATE}.json",
    orient="records", force_ascii=False, indent=2
)

print(f"✅ LLM 학습용 파일 1개 + 세부 JSON 4개가 {OUTPUT_DIR} 에 저장되었습니다.")


✅ LLM 학습용 파일 1개 + 세부 JSON 4개가 /Users/yujimin/KB AI CHALLENGE/project/llm_input 에 저장되었습니다.


In [ ]:
# # 뉴스에서 날짜만 있음 . 시간:분도 필요

# import json
# import pandas as pd
# from pathlib import Path

# # ──────────────────────────────────────────────────────────────────────────
# # 0) 공통 설정
# # ──────────────────────────────────────────────────────────────────────────
# BASE_DIR   = Path("/Users/yujimin/KB AI CHALLENGE/project")
# DATA_DIR   = BASE_DIR / "data"
# OUTPUT_DIR = BASE_DIR / "llm_input"
# OUTPUT_DIR.mkdir(exist_ok=True)

# TARGET_USER = 2
# TARGET_DATE = pd.to_datetime("2025-08-05").date()   # YYYY-MM-DD

# # ──────────────────────────────────────────────────────────────────────────
# # 1) 사용자 프로필
# # ──────────────────────────────────────────────────────────────────────────
# users_df       = pd.read_json(DATA_DIR / "dummy_users.json")
# user_profile   = users_df.loc[users_df["userId"] == TARGET_USER].iloc[0].to_dict()

# # ──────────────────────────────────────────────────────────────────────────
# # 2) 거래 내역
# # ──────────────────────────────────────────────────────────────────────────
# trades_df            = pd.read_json(DATA_DIR / "dummy_trade_history_v2.json")
# trades_df["order_date"] = pd.to_datetime(trades_df["orderTime"]).dt.date

# user_trades = trades_df.loc[
#     (trades_df["userId"] == TARGET_USER) & (trades_df["order_date"] == TARGET_DATE)
# ].copy()

# # 문자열 변환(ISO8601) → JSON 직렬화 시 pandas Timestamp 문제 방지
# user_trades["orderTime"]     = user_trades["orderTime"].astype(str)
# user_trades["completionTime"] = user_trades["completionTime"].astype(str)

# # ──────────────────────────────────────────────────────────────────────────
# # 3) 투자 일지
# # ──────────────────────────────────────────────────────────────────────────
# journals_df = pd.read_json(DATA_DIR / "dummy_trade_journals_v2.json")

# # orderTime 붙이기
# journals_with_time = journals_df.merge(
#     trades_df[["id", "orderTime"]], left_on="tradeId", right_on="id", how="left"
# )
# journals_with_time["orderTime"] = pd.to_datetime(journals_with_time["orderTime"])

# user_journals = journals_with_time.loc[
#     (journals_with_time["userId"] == TARGET_USER) &
#     (journals_with_time["orderTime"].dt.date == TARGET_DATE)
# ].copy()

# user_journals["orderTime"] = user_journals["orderTime"].astype(str)

# # ──────────────────────────────────────────────────────────────────────────
# # 4) 8월 5일 뉴스
# # ──────────────────────────────────────────────────────────────────────────
# news_df   = pd.read_csv(BASE_DIR / "news_clean_dedup.csv")
# news_0805 = news_df.loc[news_df["date"] == "2025.08.05"].copy()

# # ──────────────────────────────────────────────────────────────────────────
# # 5) 학습용 단일 JSON 레코드 구성 & 저장(JSONL 한 줄)
# # ──────────────────────────────────────────────────────────────────────────
# train_record = {
#     "meta": {
#         "user_id": TARGET_USER,
#         "date": TARGET_DATE.isoformat()
#     },
#     "user_profile": user_profile,
#     "trades":  user_trades.drop(columns=["order_date"]).to_dict(orient="records"),
#     "journals": user_journals[
#         ["journalId", "tradeId", "orderTime", "reason", "emotion", "behavior", "resultType"]
#     ].to_dict(orient="records"),
#     "news": news_0805.to_dict(orient="records")
# }

# with open(OUTPUT_DIR / f"train_user{TARGET_USER}_{TARGET_DATE}.jsonl", "w", encoding="utf-8") as f:
#     f.write(json.dumps(train_record, ensure_ascii=False) + "\n")

# # ──────────────────────────────────────────────────────────────────────────
# # 6) 파트별 개별 JSON 저장(선택)
# # ──────────────────────────────────────────────────────────────────────────
# with open(OUTPUT_DIR / f"user{TARGET_USER}_profile.json", "w", encoding="utf-8") as f:
#     json.dump(user_profile, f, ensure_ascii=False, indent=2)

# user_trades.drop(columns=["order_date"]).to_json(
#     OUTPUT_DIR / f"trades_user{TARGET_USER}_{TARGET_DATE}.json",
#     orient="records", force_ascii=False, indent=2
# )

# user_journals[
#     ["journalId", "tradeId", "orderTime", "reason", "emotion", "behavior", "resultType"]
# ].to_json(
#     OUTPUT_DIR / f"journals_user{TARGET_USER}_{TARGET_DATE}.json",
#     orient="records", force_ascii=False, indent=2
# )

# news_0805.to_json(
#     OUTPUT_DIR / f"news_{TARGET_DATE}.json",
#     orient="records", force_ascii=False, indent=2
# )

# print(f"✅ LLM 학습용 파일 1개 + 세부 JSON 4개가 {OUTPUT_DIR} 에 저장되었습니다.")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/yujimin/KB AI CHALLENGE/project/news_clean_dedup.csv'

### `train_user2_2025-08-05.jsonl` 구조 요약

> **목적**
> 하루 · 단일 사용자 · 여러 데이터 소스를 **한 줄(=하나의 JSON 객체)** 로 묶어 LLM 학습(Instruction-tuning / RAG 전처리 등)에 바로 넣을 수 있게 만든 파일입니다.
>
> **형식**: **JSON Lines** – 행(줄)마다 완전한 JSON 객체 1개. 샤딩·스트리밍 학습에 유리합니다.

```jsonc
{
  "meta": {                    // → 인덱싱·파티셔닝용 최소 메타
    "user_id": 2,
    "date": "2025-08-05"
  },
  "user_profile": { … },       // → KB KYC Enum 기반 1:1 프로필
  "trades": [ … ],             // → 당일 체결 모든 거래 (N≤∞)
  "journals": [ … ],           // → 거래별 일지 (N≤∞)  *orderTime 포함*
  "news": [ … ]                // → 당일 해당 종목 뉴스 (중복 제거 후)
}
```

| 키              | 설명                                                      | 특징                                                          |
| -------------- | ------------------------------------------------------- | ----------------------------------------------------------- |
| `meta`         | 샘플 식별자                                                  | `date`는 ISO-8601, 문자열.                                      |
| `user_profile` | **dummy\_users.json** 한 행 그대로                           | Enum 이름 그대로 보존 → 추론용 one-hot/embedding 처리 용이.               |
| `trades`       | **dummy\_trade\_history\_v2.json** 중 필터된 레코드 배열         | 시간 컬럼을 `str`(ISO) 변환해서 JSON 직렬화 문제 없음.                      |
| `journals`     | **dummy\_trade\_journals\_v2.json** + `orderTime` 조인 결과 | 감정·행동 태그가 이미 Enum 문자열로 정규화.                                 |
| `news`         | **news\_clean\_dedup.csv** 필터링 결과                       | 중복 뉴스 제거 후 종목별 포함. `text`, `link`, `stock_name`, `date` 필드. |

#### 사용 시나리오 예

1. **RAG**

   * `meta.user_id + meta.date`를 key 로 검색 → 해당 레코드에서 컨텍스트 바로 추출.
2. **Fine-tuning**

   * prompt 템플릿에 `user_profile`, `trades`, `journals`, `news`를 삽입해 “일간 리포트” 생성 supervised fine-tune.
3. **멀티태스크 학습**

   * loss 분리: sentiment, behavior 분류 추가 학습 → `journals`의 `emotion`, `behavior` 태그 활용.

#### 파일 크기·배치

* **한 줄 = 하루** 이므로 **데이터 볼륨 = 일수 × 사용자 수**.
* 수십만 행도 스트리밍 학습 가능(예: HF `datasets.load_dataset("json", split="train")` 로 바로 로드).

필요 시 키 이름·필드 subset 조정만 하면 다른 날짜·사용자도 동일 스키마로 확장 가능합니다.

---

아래는 4개의 개별 데이터 출력하는 코드. 위는 llm input 형태로 저장을 위한 통합본

In [ ]:
## user2 info 출력
import pandas as pd
df = pd.read_json("/Users/yujimin/KB AI CHALLENGE/project/data/dummy_users.json")
print(df.loc[df["userId"] == 2].to_json(orient="records", force_ascii=False, indent=2))

[
  {
    "userId":2,
    "profile":"NEUTRAL",
    "residencyCountry":"SOUTH_KOREA",
    "job":"PROFESSIONAL",
    "incomeSource":"INTEREST_DIVIDEND",
    "usagePurpose":"ASSET_GROWTH",
    "financialAssetRatio":"OVER_40_PERCENT",
    "investmentExperience":"STOCKS_AND_HIGH_RISK_FUNDS",
    "investmentDuration":"LESS_THAN_5_YEARS",
    "investmentGoal":"TRACK_MARKET_RETURN",
    "disposalPurpose":"ASSET_GROWTH",
    "ageGroup":"AGE_20_TO_40",
    "investmentPeriod":"BETWEEN_1_AND_2_YEARS",
    "incomeStability":"STABLE_BUT_DECREASING",
    "annualIncome":"OVER_100_MILLION",
    "financialKnowledgeLevel":"DEEP_UNDERSTANDING",
    "expectedReturn":"AROUND_20_PERCENT",
    "lossToleranceLevel":"PARTIAL_LOSS_ACCEPTABLE"
  }
]


In [9]:
## user2의 8월 5일 거래 기록

import pandas as pd
from pathlib import Path

# --- 1) 파일 불러오기 ------------------------------------------------------
file_path = Path("/Users/yujimin/KB AI CHALLENGE/project/data/dummy_trade_history_v2.json")  # 필요하면 경로 수정
df = pd.read_json(file_path)

# --- 2) 날짜 컬럼(YYYY-MM-DD) 생성 ----------------------------------------
df["order_date"] = pd.to_datetime(df["orderTime"]).dt.date

# --- 3) 조건 필터링 --------------------------------------------------------
target_user   = 2
target_date   = pd.to_datetime("2025-08-05").date()  # 원하는 날짜

filtered = df.loc[(df["userId"] == target_user) & (df["order_date"] == target_date)]

print(f"\n[결과] userId={target_user}, date={target_date} — {len(filtered)}건")
print(filtered.to_string(index=False))

# --- (선택) CSV나 JSON으로 저장 -------------------------------------------
# filtered.to_csv(f"user{target_user}_{target_date}_trades.csv", index=False, encoding="utf-8-sig")
# filtered.to_json(f"user{target_user}_{target_date}_trades.json", orient="records", force_ascii=False, indent=2)



[결과] userId=2, date=2025-08-05 — 4건
 userId                            tradeUUID stockName  stockCode actionType orderType           orderTime      completionTime  marketPriceAtOrder  pricePerBuy  pricePerSell  quantity  totalAmount resultType  exchangeRate  id order_date
      2 43f6a987-eee9-41e2-b5c9-d96b1777ddec      삼성전자       5930        BUY     LIMIT 2025-08-05 09:58:30 2025-08-05 10:01:30               71067      71067.0           NaN        20      1421340       None           NaN  66 2025-08-05
      2 e9c49881-a3e6-4093-97f5-34f991bf8e35     NAVER      35420       SELL     LIMIT 2025-08-05 14:25:53 2025-08-05 14:29:53              235927          NaN      235927.0        20      4718540 BREAK_EVEN           NaN  88 2025-08-05
      2 cf603d68-8496-4c33-955c-37c39ff14b49       카카오      35720       SELL     LIMIT 2025-08-05 13:08:48 2025-08-05 13:12:48               55570          NaN       55570.0        18      1000260     PROFIT           NaN 110 2025-08-05
      2 43f98eb

In [4]:
## user2의 8월 5일 투자 일지 작성 기록

import pandas as pd

# 1) 파일 로드
journals = pd.read_json("/Users/yujimin/KB AI CHALLENGE/project/data/dummy_trade_journals_v2.json")     # journalId, tradeId, userId, reason …
trades   = pd.read_json("/Users/yujimin/KB AI CHALLENGE/project/data/dummy_trade_history_v2.json")      # id (= tradeId), orderTime …

# 2) tradeId 기준으로 거래시각(orderTime) 붙이기
j_with_time = journals.merge(
    trades[["id", "orderTime"]],          # 필요한 열만
    left_on="tradeId",
    right_on="id",
    how="left"
)

# 3) 날짜-시간 컬럼을 pandas datetime 으로 변환
j_with_time["orderTime"] = pd.to_datetime(j_with_time["orderTime"])

# 4) user 2의 2025-08-05 기록만 필터링
target_date = pd.to_datetime("2025-08-05").date()
mask = (
    (j_with_time["userId"] == 2) &
    (j_with_time["orderTime"].dt.date == target_date)
)

result = j_with_time.loc[
    mask,
    ["journalId", "tradeId", "orderTime", "reason", "emotion", "behavior", "resultType"]
].sort_values("orderTime")

# 5) 원하는 형태로 출력 / 가공
print(result.to_string(index=False))


 journalId  tradeId           orderTime              reason     emotion        behavior resultType
        66       66 2025-08-05 09:58:30      뉴스를 보고 진입했습니다. EXPECTATION EMOTIONAL_ENTRY       NONE
       132      132 2025-08-05 11:32:55 평균 단가를 낮추려고 매수했습니다.   CERTAINTY     CHASING_BUY       NONE
       110      110 2025-08-05 13:08:48   분할매도 전략이 잘 먹혔습니다.         JOY  STRATEGIC_EXIT     PROFIT
        88       88 2025-08-05 14:25:53  손실 없이 나온 것에 만족합니다.      REGRET   MARKET_FOLLOW BREAK_EVEN


In [10]:
## 8월 5일 종목별 뉴스 모두 출력
import pandas as pd

# --- 1) CSV 로드 -----------------------------------------------------------
df = pd.read_csv("/Users/yujimin/KB AI CHALLENGE/project/news_clean_dedup.csv")   # 경로 필요 시 수정

# --- 2) 8월 5일(2025.08.05) 뉴스만 필터 ------------------------------------
target_day = "2025.08.05"
news_0805 = df.loc[df["date"] == target_day]

# --- 3) 종목별로 묶어서 출력 ----------------------------------------------
for stock, grp in news_0805.groupby("stock_name"):
    print(f"\n========= [{stock}] {target_day} =========")
    for i, row in grp.iterrows():
        print(f"- 기사 {i+1}: {row['text'][:120]}...")  # 길면 앞 120자만 미리보기
        print(f"  링크 : {row['link']}\n")

# --- (선택) 필터링된 결과를 CSV/JSON으로 저장 -----------------------------
# news_0805.to_csv(f"news_{target_day.replace('.','_')}.csv", index=False, encoding="utf-8-sig")
# news_0805.to_json(f"news_{target_day.replace('.','_')}.json", orient="records", force_ascii=False, indent=2)




========= [NAVER] 2025.08.05 =========
- 기사 3527: [데일리안 오늘뉴스 종합] 이춘석 '주식 차명거래 의혹'...난처한 與 "... [SEP] 이춘석 국회 법제사법위원장이 지난 1일 오후 국회에서 열린 법제사법위원회 전체회의 정회 후 박형수 법사위 국민의힘 간사를 비...
  링크 : https://finance.naver.com/item/news_read.naver?article_id=0002987827&office_id=119&code=035420&page=23&sm=

- 기사 3528: “낯익은 얼굴” 국가대표됐다더니...‘1400억 잭팟’ 또 대박 [SEP] 김성훈 업스테이지 대표. [업스테이지 유튜브 갈무리][헤럴드경제=권제인 기자] “스타트업 신화, 또 나왔다.”업스테이지가 스타트업 중 유일하...
  링크 : https://finance.naver.com/item/news_read.naver?article_id=0002510209&office_id=016&code=035420&page=23&sm=

- 기사 3529: 구글, '가림 처리' 국내 위성사진 구매 검토...보안우려 의식했나 [SEP] 정부 반출 결정 앞두고 "요구사항 이행 방안 긴밀히 협의 중"구글 "1:5000은 국가기본도"...군사시설 포함된 사실상 정밀지도 c N...
  링크 : https://finance.naver.com/item/news_read.naver?article_id=0008412352&office_id=421&code=035420&page=23&sm=

- 기사 3530: 네이버, 유럽 C2C 공략...스페인판 당근마켓 ‘왈라팝’ 인수 [SEP] 북미 포시마크·일본 소다 이어...검색·광고·AI 등 접목 시너지 기대네이버가 스페인 최대 중고거래 플랫폼인 ‘왈라팝’을 인수한다. 유럽 내...
  링크 : https://finance.naver.com/item/news_read.naver?article_id=0003387646